# Web scraping pipeline Part 3

In this section, we finally are able to apply Natural Language Processing techniques to the dataset. Firstly, we connect the scraped Data with their corresponding rows of Courses Dataset. Using regular expressions, tokenization, lemmatization we extract key words from the name, description and requirements of courses.

In [1]:
import re
import json
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Natural Language Processing packages:

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer 
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

Language detection:

In [3]:
# !{sys.executable} -m pip install langdetect
from langdetect import detect

Reading scraped Data and course Dataset:

In [4]:
df_scrapedData = pd.read_csv('../Data/interim/df_scrapedAllData.csv')

In [5]:
df_scrapedData.head()

,description,rating,audience,counter,course,requirements,language
0,Accounting is one of the most important skills...,4.6,['Aspiring Accountants and Financial Analysts'...,3663.0,640100.0,No prior knowledge of accounting is assumed or...,English
1,This course is an introduction to the financia...,3.6,"['Students in business and Finance', 'Auditors...",33.0,385604.0,some knowledge of accounting,English
2,*Course Fully Updated for May 2019*The don’t c...,4.7,['Anyone interested in earning an extra income...,300.0,834836.0,You will need some basic knowledge of stock an...,English
3,This Mortgage Acceleration course will teach y...,3.7,['This Mortgage Acceleration course is designe...,7.0,504620.0,Students will need a reliable computer and int...,English
4,"This course is for bookkeepers, accountants an...",3.9,['Individuals / Directors who want to submit t...,10.0,359926.0,It would be helpful if you understood accounti...,English


In [6]:
df_coursesSampling = pd.read_csv('../Data/interim/df_samples.csv')

In [7]:
df_coursesSampling.head()

,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,contentInfo,publishedTime,category,timeSpent,publishDate,level,paidBool
0,640100,Accounting & Financial Statement Analysis: Com...,https://www.udemy.com/accounting-fsa-a-solid-f...,True,150.0,10042,594,43,All Levels,3 hours,2015-10-22T00:03:48Z,BussinessFinance,3.0,2015-10-22,All Levels,True
1,385604,Introduction to Financial Consolidation under ...,https://www.udemy.com/introduction-to-financia...,True,25.0,21,3,8,All Levels,1.5 hours,2016-12-05T14:18:39Z,BussinessFinance,1.5,2016-12-05,All Levels,True
2,834836,How to Consistently Win Trading Stocks in 30 D...,https://www.udemy.com/winningstocktrades/,True,145.0,1433,169,15,Intermediate Level,1 hour,2016-05-09T05:44:33Z,BussinessFinance,1.0,2016-05-09,Intermediate Level,True
3,504620,Mortgage Acceleration,https://www.udemy.com/mortgage-acceleration/,True,20.0,247,2,17,All Levels,1.5 hours,2015-08-21T18:36:25Z,BussinessFinance,1.5,2015-08-21,All Levels,True
4,359926,UK Tax Returns with HMRC,https://www.udemy.com/corporation-tax-returns-...,True,40.0,2,0,11,Beginner Level,1 hour,2016-04-05T15:48:32Z,BussinessFinance,1.0,2016-04-05,Beginner Level,True


In [8]:
df_courseSampling = df_coursesSampling.join(df_scrapedData, how='outer')

In [9]:
df_courseSampling.tail()

,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,contentInfo,...,publishDate,level,paidBool,description,rating,audience,counter,course,requirements,language
1995,1011550,Build Sign Up and Login Forms With Bootstrap M...,https://www.udemy.com/build-sign-up-and-login-...,True,20.0,1898,19,31,All Levels,4 hours,...,2016-11-17,All Levels,True,Add Sign Up ModalAdding Sign Up Form To ModalS...,4.3,['This course is for anyone who wants to learn...,31.0,1011550.0,Basic HTML/CSSMYSQL insert and select queriesT...,English
1996,143028,Code a Responsive Website Using HTML5 and CSS ...,https://www.udemy.com/how-to-code-a-responsive...,True,50.0,1271,136,110,Beginner Level,7.5 hours,...,2014-01-09,Beginner Level,True,Course Overview This course is the equivalen...,4.4,['This course is designed for beginners who wa...,407.0,143028.0,Students will need to download a free copy of ...,English
1997,1179104,Learning Path: React: Make Stunning React Webs...,https://www.udemy.com/learning-path-react-make...,True,200.0,91,5,53,Expert Level,6.5 hours,...,2017-04-18,Expert Level,True,Packt’s Video Learning Paths are a series of i...,3.7,['This course is ideal for web developers. In ...,25.0,1179104.0,Requires working knowledge of ReactJS and some...,English
1998,361620,Wordpress Tutorial,https://www.udemy.com/responsive-design/,True,200.0,2311,8,44,All Levels,1.5 hours,...,2014-12-04,All Levels,True,Are you looking for a step by step video tuto...,4.2,"['Any one who is eager to succeed', 'whoever w...",11.0,361620.0,just a computer a brain and determinationyou d...,English
1999,294408,Learn to Make an Animated Image Gallery using ...,https://www.udemy.com/learn-to-make-an-animate...,False,0.0,11080,165,7,Beginner Level,1 hour,...,2014-11-20,Beginner Level,False,A short and sweet course for all the HTML5 fa...,4.5,"['Front end web develoeprs', 'Web Designers']",326.0,294408.0,"Basic Knowledge of HTML, CSS and JavaScript",English


In [10]:
df_courseSampling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 23 columns):
id                      2000 non-null int64
title                   2000 non-null object
url                     2000 non-null object
isPaid                  2000 non-null bool
price                   2000 non-null float64
numSubscribers          2000 non-null int64
numReviews              2000 non-null int64
numPublishedLectures    2000 non-null int64
instructionalLevel      2000 non-null object
contentInfo             2000 non-null object
publishedTime           2000 non-null object
category                2000 non-null object
timeSpent               2000 non-null float64
publishDate             2000 non-null object
level                   2000 non-null object
paidBool                2000 non-null bool
description             1567 non-null object
rating                  1589 non-null float64
audience                2000 non-null object
counter                 1589 non-null float

In [11]:
df_courseSampling.dropna(inplace=True)

In [12]:
df_courseSampling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1999
Data columns (total 23 columns):
id                      1460 non-null int64
title                   1460 non-null object
url                     1460 non-null object
isPaid                  1460 non-null bool
price                   1460 non-null float64
numSubscribers          1460 non-null int64
numReviews              1460 non-null int64
numPublishedLectures    1460 non-null int64
instructionalLevel      1460 non-null object
contentInfo             1460 non-null object
publishedTime           1460 non-null object
category                1460 non-null object
timeSpent               1460 non-null float64
publishDate             1460 non-null object
level                   1460 non-null object
paidBool                1460 non-null bool
description             1460 non-null object
rating                  1460 non-null float64
audience                1460 non-null object
counter                 1460 non-null float

In [13]:
df_courseSampling.description.head()

0    Accounting is one of the most important skills...
1    This course is an introduction to the financia...
2    *Course Fully Updated for May 2019*The don’t c...
3    This Mortgage Acceleration course will teach y...
4    This course is for bookkeepers, accountants an...
Name: description, dtype: object

In [14]:
df_courseSampling.description[0]

"Accounting is one of the most important skills for people pursuing a career in Finance.It helps you understand whether a business is profitable.It gives you an idea of a company’s size.It helps you use the past in order to take action in the present and change the future.However, it’s essential that you understand it well. If you want to become…a Financial Analystan Accountantan Auditora Business Analysta Financial Controllera Financial Managera CFOa CEOan Investment Bankeran Equity Research Analystan Investor an Entrepreneur Someone who is involved with a business and would like to be successfulThen you simply have to learn Accounting and Financial Statement Analysis. There is no way around it.But how can you do that if you have very limited time and no prior training? And how can you be sure that you are not missing an important piece of the puzzle?Accounting &amp; Financial Statement Analysis: Complete Training is here for you. One of the best Finance courses available on Udemy, it

## Natural Language pre-processing 

### 1. Language detection

In [15]:
def english_detection(string):
    if detect(string) == 'en':
        return True
    else:
        return False

### 2. Building **expandContractions** function

In [16]:
"""
from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
all credits go to alko and arturomp @ stack overflow.
"""

with open('../Data/nlp/wordLists/contractionsList.txt', 'r') as f:
    cList = json.loads(f.read())
    c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [17]:
wpt = nltk.WordPunctTokenizer()
lemmatizer = WordNetLemmatizer() 
stemmer = PorterStemmer() 

### 3. **Corpus normalized** of course descriptions

Let's start to normalize the course description of the row 0, whose name is *Accounting & Financial Statement Analysis: Complete Training*

In [18]:
df_courseSampling.title[0]

'Accounting & Financial Statement Analysis: Complete Training'

**Original description**:

In [19]:
df_courseSampling.description[0]

"Accounting is one of the most important skills for people pursuing a career in Finance.It helps you understand whether a business is profitable.It gives you an idea of a company’s size.It helps you use the past in order to take action in the present and change the future.However, it’s essential that you understand it well. If you want to become…a Financial Analystan Accountantan Auditora Business Analysta Financial Controllera Financial Managera CFOa CEOan Investment Bankeran Equity Research Analystan Investor an Entrepreneur Someone who is involved with a business and would like to be successfulThen you simply have to learn Accounting and Financial Statement Analysis. There is no way around it.But how can you do that if you have very limited time and no prior training? And how can you be sure that you are not missing an important piece of the puzzle?Accounting &amp; Financial Statement Analysis: Complete Training is here for you. One of the best Finance courses available on Udemy, it

Before extract meaningful words, there is a lot of work to do. We note the lack of spaces between miex words; the use of contractions and stopwords. Let's define a function that:
1. Expand contractions
2. Incorporate an extra spaces between mixed words.
3. Remove special characters (once they have been isolated)
4. Apply tokenization
5. Filtering stop-words
6. Apply lemmatization to get the root word

In [20]:
def corpus_normalization(original_text, stop_words, wordlist=False):
    """
    This function receives the origin text and return a corpus normalized. If the stop words are updated, 
    we exclude the word 'no'. In requirements, the difference between 'No previous knowlegde required' 
    and 'previous knowlegde required' is crucial. In description of courses, we are looking for key-words 
    and the edition of the stop words from nltk is not necessary. 
    """
    # Mixed words have uppercase after lowercase letters. Ex:learnedExcellent
    pattern = '[a-z][A-Z]'
    # Return empty string if input is not string, np.nan, None
    if (original_text is None) or (type(original_text) is not str) or (original_text in ['None', 'NIL']):
        return ''
    # Return empty string if input is a non-english string:
    elif english_detection(original_text) is False:
        return ''
    else:
        text = re.sub(r'’',"'", original_text)
        text = expandContractions(text)
        word_list = wpt.tokenize(text)
        # Incorporate extra spaces between mixed words
        words_edit = []
        for word in word_list:
            if re.search(pattern, word): 
                index = re.search(pattern, word).start()
                word = word[:index+1] + ' ' + word[index+1:]
            words_edit.append(word)

        text_filtered = ' '.join(words_edit)
        # Filtering special characters
        text_filtered  = re.sub(r'[^a-zA-Z\s]','', text_filtered)
        text_filtered  = text_filtered.lower()
        # Tokenization to filter stopwords and retrieve roots from derivated words
        tokens = wpt.tokenize(text_filtered)
        words_lem = [lemmatizer.lemmatize(word) for word in tokens if lemmatizer.lemmatize(word) not in stop_words and len(word) > 3]
        if wordlist:
            return words_lem
        text_norm = ' '.join(words_lem)
        
    return text_norm

In [21]:
stop_words = stopwords.words('english')
stop_words.extend(['udemy', 'course', 'school', 'lesson', 'ucstrong', 'rating', 'time', 
                   'html', 'student', 'section', 'professional', 'also', 'using', 'want', 
                   'make', 'take', 'need', 'easy', 'free', 'help', 'basic', 'lecture'])

In [22]:
df_courseSampling['normalized_descriptions'] = df_courseSampling.description.apply(corpus_normalization, stop_words=stop_words)

In [23]:
df_courseSampling['normalized_descriptions'].head()

0    accounting important skill people pursuing car...
1    introduction financial consolidation ifrs aim ...
2    fully updated call trading profit nothing lite...
3    mortgage acceleration teach mortgage work beat...
4    bookkeeper accountant limited company director...
Name: normalized_descriptions, dtype: object

### Corpus normalized of course requirements

In [24]:
to_remove = ['no']
update_stop_words = set(stopwords.words('english')).difference(to_remove)

In [25]:
df_courseSampling['normalized_requirements'] = df_courseSampling.requirements.apply(corpus_normalization, stop_words=update_stop_words)

In [26]:
df_courseSampling['normalized_requirements'].head()

0    prior knowledge accounting assumed needed noth...
1                                 knowledge accounting
2    need basic knowledge stock option trading need...
3    student need reliable computer internet connec...
4    would helpful understood accounting terminolog...
Name: normalized_requirements, dtype: object

In [27]:
df_courseSampling.loc[:, ['description', 'normalized_descriptions', 'requirements', 'normalized_requirements']].head()

,description,normalized_descriptions,requirements,normalized_requirements
0,Accounting is one of the most important skills...,accounting important skill people pursuing car...,No prior knowledge of accounting is assumed or...,prior knowledge accounting assumed needed noth...
1,This course is an introduction to the financia...,introduction financial consolidation ifrs aim ...,some knowledge of accounting,knowledge accounting
2,*Course Fully Updated for May 2019*The don’t c...,fully updated call trading profit nothing lite...,You will need some basic knowledge of stock an...,need basic knowledge stock option trading need...
3,This Mortgage Acceleration course will teach y...,mortgage acceleration teach mortgage work beat...,Students will need a reliable computer and int...,student need reliable computer internet connec...
4,"This course is for bookkeepers, accountants an...",bookkeeper accountant limited company director...,It would be helpful if you understood accounti...,would helpful understood accounting terminolog...


In [28]:
df_courseSampling.reset_index(inplace=True)

## Topic Modeling using Latent Dirichlet Allocation (LDA)

The first purpose is to recognize the topic of every course: 60% of the dataset is used to train a LDA model with the description of courses, chosing courses from the four different categories **Web Development**, **Graphic Design**, **Music and Instrument** and **Bussiness Finance** to avoid unbalanced or biased training dataset.
In this section, we use gensim pachage to generate our Bag of Words.

In [29]:
df_english = df_courseSampling[df_courseSampling.normalized_descriptions != ''].reindex()

In [30]:
df_english.groupby('category').count()

,index,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,...,paidBool,description,rating,audience,counter,course,requirements,language,normalized_descriptions,normalized_requirements
category,,,,,,,,,,,,,,,,,,,,,
BussinessFinance,327,327,327,327,327,327,327,327,327,327,...,327,327,327,327,327,327,327,327,327,327
GraphicDesign,306,306,306,306,306,306,306,306,306,306,...,306,306,306,306,306,306,306,306,306,306
MusicInstrument,369,369,369,369,369,369,369,369,369,369,...,369,369,369,369,369,369,369,369,369,369
WebDevelopment,401,401,401,401,401,401,401,401,401,401,...,401,401,401,401,401,401,401,401,401,401


In [31]:
df_train = df_english.groupby('category').apply(lambda x: x.sample(frac=0.6, random_state=1)).reset_index(drop=True)

In [32]:
df_train.groupby('category').count()

,index,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,...,paidBool,description,rating,audience,counter,course,requirements,language,normalized_descriptions,normalized_requirements
category,,,,,,,,,,,,,,,,,,,,,
BussinessFinance,196,196,196,196,196,196,196,196,196,196,...,196,196,196,196,196,196,196,196,196,196
GraphicDesign,184,184,184,184,184,184,184,184,184,184,...,184,184,184,184,184,184,184,184,184,184
MusicInstrument,221,221,221,221,221,221,221,221,221,221,...,221,221,221,221,221,221,221,221,221,221
WebDevelopment,241,241,241,241,241,241,241,241,241,241,...,241,241,241,241,241,241,241,241,241,241


In [33]:
df_train.head()

,index,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,...,paidBool,description,rating,audience,counter,course,requirements,language,normalized_descriptions,normalized_requirements
0,89,455452,"Accounting, Finance and Banking - A Comprehens...",https://www.udemy.com/accounting-finance-and-b...,True,180.0,507,36,395,All Levels,...,True,"Welcome to this dream course ""Accounting, Fina...",3.9,"['Accounting Students', 'Banking Students', 'F...",168.0,455452.0,This course will teach you from basic to advan...,English,welcome dream accounting finance banking compr...,course teach basic advanced concept hence take...
1,391,302562,Introduction to Accounting: The Language of Bu...,https://www.udemy.com/learnaccountingforfree/,True,20.0,11958,370,134,Beginner Level,...,True,Learn accounting from the self-made millionair...,4.6,"['Students', ' Entrepreneurs', ' and anyone wh...",1470.0,302562.0,"Elementary math skills (i.e., basic addition s...",English,learn accounting self made millionaire norm ne...,elementary math skill basic addition subtracti...
2,419,1273896,Covered Calls - Powerful Income Strategy for S...,https://www.udemy.com/covered-calls-income-str...,True,60.0,22,0,8,Beginner Level,...,True,If you're a Stock trader or Long term stock in...,4.2,"['All Stock Traders', 'Self-directed investors...",33.0,1273896.0,Trading Stocks and holding a stock portfolioAl...,English,stock trader long term stock investor hold sto...,trading stock holding stock portfolio although...
3,341,1045726,Stock Market investment:Non financial fundamen...,https://www.udemy.com/stock-market-investmentn...,True,125.0,1091,4,29,Beginner Level,...,True,"As an investor, how often do you findyourself ...",3.3,['Investors who wish to do their own stock mar...,28.0,1045726.0,Having some knowledge of accounting would be g...,English,investor often findyourself lost read company ...,knowledge accounting would good mandatory
4,190,42643,Master Iron Condors - Double the credit for ha...,https://www.udemy.com/iron-condor-options-trad...,True,60.0,1338,139,7,Expert Level,...,True,THE IRON CONDOR STRATEGY - KING OF TIME DECAY ...,4.4,"['This is an advanced level course', ' so stud...",290.0,42643.0,Excellent knowledge of Credit spreads and all ...,English,iron condor strategy king decay strategiesthe ...,excellent knowledge credit spread material exc...


In [34]:
df_english.groupby('category').count()

,index,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,...,paidBool,description,rating,audience,counter,course,requirements,language,normalized_descriptions,normalized_requirements
category,,,,,,,,,,,,,,,,,,,,,
BussinessFinance,327,327,327,327,327,327,327,327,327,327,...,327,327,327,327,327,327,327,327,327,327
GraphicDesign,306,306,306,306,306,306,306,306,306,306,...,306,306,306,306,306,306,306,306,306,306
MusicInstrument,369,369,369,369,369,369,369,369,369,369,...,369,369,369,369,369,369,369,369,369,369
WebDevelopment,401,401,401,401,401,401,401,401,401,401,...,401,401,401,401,401,401,401,401,401,401


In [35]:
print('Number of documents: {}'.format(len(df_train)))

Number of documents: 842


In [36]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [37]:
# !{sys.executable} -m pip install pyLDAvis
# !{sys.executable} -m pip install -U gensim

In [38]:
# !{sys.executable} -m pip install paramiko

In [39]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, TfidfModel, LdaMulticore

In [40]:
# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [41]:
df_train['wordList_description'] = df_train.description.apply(corpus_normalization, stop_words=stop_words, wordlist=True)

In [42]:
data_words = list(df_train['wordList_description'])

In [43]:
data_words[1]

['learn',
 'accounting',
 'self',
 'made',
 'millionaire',
 'norm',
 'nemrow',
 'recipient',
 'famed',
 'teaching',
 'award',
 'president',
 'united',
 'state',
 'produced',
 'accounting',
 'university',
 'world',
 'brigham',
 'young',
 'university',
 'rated',
 'london',
 'financial',
 'teach',
 'fundamental',
 'financial',
 'accounting',
 'better',
 'effectively',
 'available',
 'today',
 'guaranteed',
 'talk',
 'good',
 'game',
 'gold',
 'standard',
 'specific',
 'prepares',
 'recruit',
 'four',
 'accounting',
 'firm',
 'worldwide',
 'recommended',
 'harvard',
 'incoming',
 'join',
 'university',
 'world',
 'famous',
 'making',
 'available',
 'accounting',
 'knowledge',
 'highlighted',
 'wired',
 'magazine',
 'gagaom',
 'york',
 'popular',
 'well',
 'acclaimed',
 'welcome',
 'norm',
 'nemrow',
 'accounting',
 'nnac',
 'accounting',
 'introduction',
 'accounting',
 'series',
 'composed',
 'five',
 'challenge',
 'learn',
 'content',
 'five',
 'mastered',
 'first',
 'year',
 'accounting

### Bag of words: 

Then, a dictionary from the corpus normalized is created to count the number of times a word appears in the training set:

In [44]:
# Create Dictionary
dictionary = corpora.Dictionary(data_words)

Filtering extremes tokens:
Let's filter tokens that appears in less than 5% of the documents and more than 50% of them.

In [45]:
dictionary.filter_extremes(no_below=40, no_above=0.6)

In [46]:
# Create Corpus
texts = data_words

Using **gensim.doc2bow** we create a dictionary for each document reporting how many words and times the words appear there. This is called corpus and the first corpus is printed as index and then as readable words:

In [47]:
# Term Document Frequency
bow_corpus = [dictionary.doc2bow(text) for text in texts]

print(bow_corpus[:1])

[[(0, 1), (1, 1), (2, 16), (3, 2), (4, 5), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 6), (19, 5), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 4), (30, 2), (31, 1), (32, 1), (33, 1), (34, 3), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 1), (45, 2), (46, 2), (47, 1), (48, 1), (49, 1), (50, 1), (51, 2), (52, 5), (53, 2), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1)]]


In [48]:
# Human readable format of corpus (term-frequency)
[[(dictionary[id], freq) for id, freq in cp] for cp in bow_corpus[:1]]

[[('access', 1),
  ('account', 1),
  ('accounting', 16),
  ('advanced', 2),
  ('analysis', 5),
  ('answer', 1),
  ('aspect', 1),
  ('beginner', 1),
  ('best', 1),
  ('business', 1),
  ('cannot', 1),
  ('case', 1),
  ('complete', 1),
  ('comprehensive', 1),
  ('concept', 2),
  ('content', 1),
  ('cost', 1),
  ('curriculum', 1),
  ('finance', 6),
  ('financial', 5),
  ('function', 2),
  ('gain', 1),
  ('going', 1),
  ('individual', 1),
  ('introduction', 1),
  ('investment', 1),
  ('knowledge', 1),
  ('learning', 1),
  ('list', 1),
  ('management', 4),
  ('minute', 2),
  ('money', 1),
  ('name', 1),
  ('practical', 1),
  ('process', 3),
  ('purpose', 1),
  ('question', 1),
  ('reading', 1),
  ('right', 1),
  ('self', 1),
  ('single', 1),
  ('skill', 1),
  ('standard', 2),
  ('statement', 1),
  ('strong', 1),
  ('structured', 2),
  ('study', 2),
  ('style', 1),
  ('sure', 1),
  ('system', 1),
  ('teach', 1),
  ('term', 2),
  ('three', 5),
  ('type', 2),
  ('value', 1),
  ('video', 1),
  (

### TF-IDF

TF-IDF model from gensim.models:

In [49]:
tfidf = TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [50]:
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus_tfidf[:1]]

[[('access', 0.028128441834495345),
  ('account', 0.04681172022692055),
  ('accounting', 0.8001056533281762),
  ('advanced', 0.05984670406573766),
  ('analysis', 0.22344278236163698),
  ('answer', 0.04255091060665999),
  ('aspect', 0.048511279017511684),
  ('beginner', 0.023186062101647777),
  ('best', 0.023587305879245526),
  ('business', 0.03153748473142274),
  ('cannot', 0.04526831448390567),
  ('case', 0.04681172022692055),
  ('complete', 0.025751125799318476),
  ('comprehensive', 0.04385476661067422),
  ('concept', 0.053993956709071554),
  ('content', 0.025385648009085948),
  ('cost', 0.04134094134411088),
  ('curriculum', 0.048872826634439974),
  ('finance', 0.2681313388339644),
  ('financial', 0.18165207479915726),
  ('function', 0.0841128489616676),
  ('gain', 0.04358577981247303),
  ('going', 0.03153748473142274),
  ('individual', 0.048157343497717046),
  ('introduction', 0.03777744242746399),
  ('investment', 0.044128127369153086),
  ('knowledge', 0.023506287330956326),
  ('l

### Running LDA using Bow and TFIDF corpus

In [69]:
ldaMulticore_model = LdaMulticore(bow_corpus, num_topics=4, id2word=dictionary, passes=20, workers=2, random_state=1)

In [70]:
ldaMulticore_model.print_topics()

[(0,
  '0.038*"music" + 0.034*"play" + 0.028*"chord" + 0.022*"guitar" + 0.019*"playing" + 0.017*"song" + 0.017*"piano" + 0.015*"video" + 0.012*"practice" + 0.011*"scale"'),
 (1,
  '0.045*"design" + 0.021*"create" + 0.020*"photoshop" + 0.017*"tool" + 0.017*"graphic" + 0.016*"project" + 0.015*"image" + 0.015*"logo" + 0.013*"illustrator" + 0.011*"work"'),
 (2,
  '0.021*"website" + 0.015*"create" + 0.013*"code" + 0.013*"development" + 0.012*"build" + 0.012*"application" + 0.011*"project" + 0.009*"page" + 0.009*"developer" + 0.009*"data"'),
 (3,
  '0.032*"trading" + 0.020*"market" + 0.019*"financial" + 0.018*"stock" + 0.017*"money" + 0.016*"option" + 0.015*"strategy" + 0.015*"trade" + 0.012*"script" + 0.012*"trader"')]

Let's calculate the perplexity and coherence score, two values used to evaluate quantitatively topic models. 

In [71]:
# Compute Perplexity
print('\nPerplexity: ', ldaMulticore_model.log_perplexity(bow_corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldaMulticore_model, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.755147174202338

Coherence Score:  0.5680237087548101


In [72]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldaMulticore_model, bow_corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.079952 -0.025918       1        1  32.823635
0     -0.166274 -0.010406       2        1  27.983639
1      0.048458 -0.097551       3        1  19.608814
3      0.037864  0.133875       4        1  19.583912, topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
451   Default  774.000000       design  774.000000  30.0000  30.0000
502   Default  821.000000        music  821.000000  29.0000  29.0000
380   Default  754.000000         play  754.000000  28.0000  28.0000
198   Default  486.000000      trading  486.000000  27.0000  27.0000
494   Default  613.000000        chord  613.000000  26.0000  26.0000
504   Default  482.000000       guitar  482.000000  25.0000  25.0000
492   Default  406.000000      playing  406.000000  24.0000  24.0000
471   Default  559.000000      website  559.000000  23.0000  23.0000
497   Default  300.000000    photoshop  300.000000  22.0000  22.0000
87    Default  321.000000       market  321.000000  21.0000  21.0000
481   Default  368.000000         song  368.000000  20.0000  20.0000
507   Default  364.000000        piano  364.000000  19.0000  19.0000
136   Default  273.000000        stock  273.000000  18.0000  18.0000
19    Default  356.000000    financial  356.000000  17.0000  17.0000
431   Default  268.000000      graphic  268.000000  16.0000  16.0000
180   Default  256.000000       option  256.000000  15.0000  15.0000
319   Default  538.000000      project  538.000000  14.0000  14.0000
138   Default  228.000000        trade  228.000000  13.0000  13.0000
137   Default  242.000000     strategy  242.000000  12.0000  12.0000
477   Default  237.000000         logo  237.000000  11.0000  11.0000
279   Default  772.000000       create  772.000000  10.0000  10.0000
157   Default  286.000000        image  286.000000   9.0000   9.0000
423   Default  356.000000         code  356.000000   8.0000   8.0000
498   Default  192.000000  illustrator  192.000000   7.0000   7.0000
139   Default  180.000000       trader  180.000000   6.0000   6.0000
110   Default  386.000000         tool  386.000000   5.0000   5.0000
466   Default  393.000000       script  393.000000   4.0000   4.0000
404   Default  249.000000        scale  249.000000   3.0000   3.0000
508   Default  383.000000         java  383.000000   2.0000   2.0000
185   Default  359.000000  development  359.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
276    Topic4   71.353760       return   86.642784   1.4363  -5.3623
19     Topic4  287.336548    financial  356.837646   1.4138  -3.9693
278    Topic4  115.931686    technical  144.957489   1.4070  -4.8770
18     Topic4  111.667053      finance  146.531036   1.3587  -4.9144
51     Topic4   56.896011         term   77.058250   1.3271  -5.5887
182    Topic4  118.924713        price  181.881012   1.2056  -4.8515
307    Topic4   80.922089      success  125.915848   1.1883  -5.2365
73     Topic4  107.388443       future  170.024948   1.1710  -4.9535
16     Topic4   75.494728         cost  123.258400   1.1402  -5.3059
106    Topic4   64.915710   successful  107.656082   1.1246  -5.4569
31     Topic4  264.464478        money  466.383179   1.0632  -4.0522
37     Topic4   52.821182      reading   94.929199   1.0442  -5.6630
131    Topic4   37.010796    portfolio   66.780624   1.0403  -6.0187
54     Topic4  105.925041        value  192.771286   1.0317  -4.9672
188    Topic4   28.986357       highly   52.901619   1.0289  -6.2631
143    Topic4   38.092155       action   71.919327   0.9949  -5.9899
125    Topic4   58.955086         long  111.886780   0.9898  -5.5532
466    Topic4  189.336060       script  393.921051   0.8978  -4.3864
177    Topic4   52.324245       income  100.476181   0.9780  -5.6725
508    Topic4  170.337067         java  383.7

Save the model to disk importing `datapath`:

In [73]:
from gensim.test.utils import datapath

temp_file = datapath('gensim_ldaMulticoreModel.csv')
coherence_model_lda.save(temp_file)

In [74]:
# lda_model = gensim.models.ldamodel.LdaModel(corpus=bow_corpus,
#            id2word=dictionary,
#            num_topics=4, 
#            random_state=100,
#            update_every=1,
# #            chunksize=300,
#            passes=2,
#            alpha='auto',
#            per_word_topics=True)

In [75]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=4, id2word=dictionary, passes=20, workers=4, random_state=0)

In [76]:
lda_model_tfidf.print_topics()

[(0,
  '0.050*"financial" + 0.042*"accounting" + 0.026*"finance" + 0.024*"business" + 0.023*"analysis" + 0.023*"statement" + 0.018*"management" + 0.015*"company" + 0.014*"model" + 0.012*"account"'),
 (1,
  '0.013*"design" + 0.012*"website" + 0.010*"create" + 0.009*"photoshop" + 0.008*"project" + 0.007*"application" + 0.007*"code" + 0.007*"graphic" + 0.007*"illustrator" + 0.007*"image"'),
 (2,
  '0.077*"script" + 0.076*"java" + 0.034*"server" + 0.022*"javascript" + 0.016*"data" + 0.015*"object" + 0.015*"programming" + 0.012*"database" + 0.010*"language" + 0.009*"side"'),
 (3,
  '0.019*"music" + 0.017*"guitar" + 0.016*"play" + 0.016*"trading" + 0.014*"chord" + 0.013*"piano" + 0.011*"playing" + 0.011*"song" + 0.009*"stock" + 0.008*"trade"')]

In [77]:
# Compute Perplexity
print('\nPerplexity: ', lda_model_tfidf.log_perplexity(corpus_tfidf))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_lda_model_tfidf = CoherenceModel(model=lda_model_tfidf, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda_model_tfidf = coherence_lda_model_tfidf.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_tfidf)


Perplexity:  -6.778149884442168

Coherence Score:  0.5413349817189845


In [78]:
pyLDAvis.enable_notebook()
vis_tfidf = pyLDAvis.gensim.prepare(lda_model_tfidf, corpus_tfidf, dictionary)
vis_tfidf

/Users/daniela/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.055034 -0.001487       1        1  50.178516
3      0.065924  0.105191       2        1  36.633595
0      0.082744 -0.094834       3        1   7.953820
2     -0.093634 -0.008870       4        1   5.234073, topic_info=     Category       Freq           Term      Total  loglift  logprob
term                                                                
466   Default  22.000000         script  22.000000  30.0000  30.0000
508   Default  22.000000           java  22.000000  29.0000  29.0000
19    Default  21.000000      financial  21.000000  28.0000  28.0000
2     Default  16.000000     accounting  16.000000  27.0000  27.0000
463   Default  12.000000         server  12.000000  26.0000  26.0000
9     Default  17.000000       business  17.000000  25.0000  25.0000
509   Default  11.000000     javascript  11.000000  24.0000  24.0000
502   Default  32.000000          music  32.000000  23.0000  23.0000
18    Default  10.000000        finance  10.000000  22.0000  22.0000
246   Default  14.000000           data  14.000000  21.0000  21.0000
4     Default  11.000000       analysis  11.000000  20.0000  20.0000
504   Default  29.000000         guitar  29.000000  19.0000  19.0000
499   Default  10.000000         object  10.000000  18.0000  18.0000
457   Default  10.000000    programming  10.000000  17.0000  17.0000
43    Default   9.000000      statement   9.000000  16.0000  16.0000
198   Default  27.000000        trading  27.000000  15.0000  15.0000
380   Default  28.000000           play  28.000000  14.0000  14.0000
29    Default   9.000000     management   9.000000  13.0000  13.0000
65    Default  11.000000        company  11.000000  12.0000  12.0000
494   Default  24.000000          chord  24.000000  11.0000  11.0000
82    Default  11.000000       language  11.000000  10.0000  10.0000
345   Default   9.000000          model   9.000000   9.0000   9.0000
257   Default   8.000000       database   8.000000   8.0000   8.0000
267   Default  11.000000           user  11.000000   7.0000   7.0000
423   Default  18.000000           code  18.000000   6.0000   6.0000
507   Default  22.000000          piano  22.000000   5.0000   5.0000
1     Default   9.000000        account   9.000000   4.0000   4.0000
451   Default  30.000000         design  30.000000   3.0000   3.0000
405   Default  19.000000    application  19.000000   2.0000   2.0000
170   Default   9.000000           risk   9.000000   1.0000   1.0000
...       ...        ...            ...        ...      ...      ...
457    Topic4   3.560611    programming  10.599309   1.8591  -4.2022
246    Topic4   3.796610           data  14.144587   1.6348  -4.1380
285    Topic4   1.520259        dynamic   6.199253   1.5444  -5.0533
82     Topic4   2.336165       language  11.630942   1.3448  -4.6236
470    Topic4   1.309654          write   7.113527   1.2577  -5.2024
261    Topic4   1.035980          field   5.931315   1.2051  -5.4368
267    Topic4   1.809172           user  11.941677   1.0628  -4.8793
121    Topic4   0.640423           hold   4.827299   0.9301  -5.9178
20     Topic4   1.166640       function   9.053577   0.9009  -5.3180
345    Topic4   1.234775          model   9.766409   0.8819  -5.2613
209    Topic4   0.691283           core   5.730381   0.8350  -5.8413
490    Topic4   0.676751        library   5.694061   0.8201  -5.8626
43     Topic4   1.151047      statement   9.889498   0.7992  -5.3315
321    Topic4   0.644300           quiz   6.056210   0.7093  -5.9117
128    Topic4   0.420717          often   4.443627   0.5927  -6.3379
274    Topic4   1.007663           form  10.682118   0.5890  -5.4645
320    Topic4   1.083490          query  11.784287   0.5634  -5.3920
318    Topic4   0.461877         number   5.233234   0.5225  -6.2446
66     Topic4   0.396296      developed   4.626485   0.4926  -6.3977
266    Topic4   0.597928       resource   7.1

In [61]:
# temp_file_tfidf = datapath('gensim_ldaMulticoreTFIDFModel.csv')
# coherence_model_lda.save(temp_file_tfidf)

### Performance evaluation Bag of words

In [62]:
def test_generator(id_course):
    if id_course in list(df_train.id):
        return 'train'
    else:
        return 'test'

In [63]:
df_english['type_data'] = df_english.id.apply(test_generator)
df_test = df_english[df_english.type_data == 'test']

In [64]:
df_test.groupby('category').count()

,index,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,...,description,rating,audience,counter,course,requirements,language,normalized_descriptions,normalized_requirements,type_data
category,,,,,,,,,,,,,,,,,,,,,
BussinessFinance,131,131,131,131,131,131,131,131,131,131,...,131,131,131,131,131,131,131,131,131,131
GraphicDesign,122,122,122,122,122,122,122,122,122,122,...,122,122,122,122,122,122,122,122,122,122
MusicInstrument,148,148,148,148,148,148,148,148,148,148,...,148,148,148,148,148,148,148,148,148,148
WebDevelopment,160,160,160,160,160,160,160,160,160,160,...,160,160,160,160,160,160,160,160,160,160


In [65]:
df_test['wordList_description'] = df_test.description.apply(corpus_normalization, stop_words=stop_words, wordlist=True)

/Users/daniela/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [66]:
test_words = list(df_test['wordList_description'])

In [67]:
bow_tests = [dictionary.doc2bow(text) for text in test_words]

Let's analyse an first course on the test dataset:

In [ ]:
df_test.reset_index(inplace=True)

In [86]:
print('Title: {}'.format(df_test.title[0]))
print('Category: {}'.format(df_test.category[0]))

Title: Introduction to Financial Consolidation under IFRS
Category: BussinessFinance


Likely to belongs to each category:

In [90]:
for index, score in sorted(ldaMulticore_model[bow_tests[0]], key=lambda tup: -1*tup[1]):
    print('Score: {}\t Topic: {}'.format(score, ldaMulticore_model.print_topic(index, 5)))

Score: 0.9629970192909241	 Topic: 0.032*"trading" + 0.020*"market" + 0.019*"financial" + 0.018*"stock" + 0.017*"money"
Score: 0.012487780302762985	 Topic: 0.038*"music" + 0.034*"play" + 0.028*"chord" + 0.022*"guitar" + 0.019*"playing"
Score: 0.012320054695010185	 Topic: 0.045*"design" + 0.021*"create" + 0.020*"photoshop" + 0.017*"tool" + 0.017*"graphic"
Score: 0.012195125222206116	 Topic: 0.021*"website" + 0.015*"create" + 0.013*"code" + 0.013*"development" + 0.012*"build"


In [187]:
print('Title: {}'.format(df_test.title[150]))
print('Category: {}'.format(df_test.category[150]))

Title: Drawing Perspective For Beginners
Category: GraphicDesign


In [188]:
for index, score in sorted(ldaMulticore_model[bow_tests[150]], key=lambda tup: -1*tup[1]):
    print('Score: {}\t Topic: {}'.format(score, ldaMulticore_model.print_topic(index, 5)))

Score: 0.9464473724365234	 Topic: 0.045*"design" + 0.021*"create" + 0.020*"photoshop" + 0.017*"tool" + 0.017*"graphic"
Score: 0.04527493938803673	 Topic: 0.038*"music" + 0.034*"play" + 0.028*"chord" + 0.022*"guitar" + 0.019*"playing"


In [189]:
print('Title: {}'.format(df_test.title[300]))
print('Category: {}'.format(df_test.category[300]))

Title: Learn BASS GUITAR- Have Fun, Amaze Your Friends- It's Easy! 
Category: MusicInstrument


In [190]:
for index, score in sorted(ldaMulticore_model[bow_tests[300]], key=lambda tup: -1*tup[1]):
    print('Score: {}\t Topic: {}'.format(score, ldaMulticore_model.print_topic(index, 5)))

Score: 0.9920145273208618	 Topic: 0.038*"music" + 0.034*"play" + 0.028*"chord" + 0.022*"guitar" + 0.019*"playing"


In [179]:
df_predicted_topics = pd.DataFrame()

In [180]:
topic_num_list = []
topic_prob_list = []
topic_key_words = []

for i in range(len(bow_tests)):
    for index, score in enumerate(ldaMulticore_model[bow_tests[i]]):
        row = sorted(ldaMulticore_model[bow_tests[i]], key=lambda tup: -1*tup[1])
        for j, (topic_num, topic_prob) in enumerate(row):
            if j==0:
                wp = ldaMulticore_model.show_topic(topic_num)
                topic_keywords = ', '.join([word for word, prop in wp])
                topic_num_list.append(int(topic_num))
                topic_prob_list.append(round(topic_prob, 4))
                topic_key_words.append(topic_keywords)
#                 df_predicted_topics.append(pd.Series([int(topic_num), round(topic_prob, 4), topic_keywords]), ignore_index=True)       
            else:
                continue

In [182]:
df_predicted_topics = pd.DataFrame.from_dict({'topic_num': topic_num_list, 
                                    'topic_prob': topic_prob_list, 'topic_keywords': topic_key_words})

In [184]:
category = pd.Series(df_test.category)
title = pd.Series(df_test.title)
df_predicted_topics = pd.concat([title, category, df_predicted_topics], axis=1)